In [1]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
import optuna
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict

# Add parent directory to path to access custom modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.modelling_functions import create_target_variable, check_feature_stability, detect_market_regimes, regime_aware_validation, calculate_sharpe_with_costs, calculate_max_drawdown

In [2]:
# --- CONFIGURATION ---
CONFIG = {
    'data_path': r'C:\Users\epoch_bpjmdqk\Documents\Code\data\processed\consumer_staples_data.csv',
    'model_dir': r"C:\Users\epoch_bpjmdqk\Documents\Code\models",
    'window': 5,
    'threshold': 0.005,
    'target_ticker': 'WMT',
    'train_end': '2020-12-31',
    'val_end': '2021-12-31',
    'test_start': '2022-01-01',
    'n_trials': 100,
    'transaction_cost': 0.001,  # 0.1% per trade
    'random_state': 42
}

In [3]:
# --- 1. DATA PREPARATION ---
try:
    data = pd.read_csv(CONFIG['data_path'], index_col='Date', parse_dates=True)
    print(f"✅ Data loaded successfully: {data.shape}")
except FileNotFoundError:
    print(f"❌ Error: Data file not found at {CONFIG['data_path']}")
    sys.exit()

# Create target variable
data_target = create_target_variable(data.copy(), CONFIG['target_ticker'], 
                                   window=CONFIG['window'], threshold=CONFIG['threshold'])
target_return_col = f"{CONFIG['target_ticker']}_target_return_{CONFIG['window']}D_{CONFIG['threshold']}"

# Define features (exclude target-related columns)
exclude_cols = [
    f"{CONFIG['target_ticker']}_Target",
    target_return_col,
    f"Open_{CONFIG['target_ticker']}",
    f"High_{CONFIG['target_ticker']}",
    f"Low_{CONFIG['target_ticker']}",
    f"Close_{CONFIG['target_ticker']}",
    f"Volume_{CONFIG['target_ticker']}",
    f"Dividends_{CONFIG['target_ticker']}",
    f"Stock Splits_{CONFIG['target_ticker']}"
]

features = [col for col in data_target.columns if col not in exclude_cols]
data_target.dropna(inplace=True)

X_features = data_target[features]
y = data_target[f"{CONFIG['target_ticker']}_Target"]
returns_full = data_target[target_return_col]

# Clean data
X_features.replace([np.inf, -np.inf], np.nan, inplace=True)
X_features = X_features.fillna(X_features.mean()).fillna(0)

# Calculate class imbalance
neg_to_pos_ratio = (y == 0).sum() / (y == 1).sum()

print(f"\n--- Data Setup ---")
print(f"Window: {CONFIG['window']}, Threshold: {CONFIG['threshold']}")
print(f"Features: {len(features)}, Samples: {len(X_features)}")
print(f"Class imbalance ratio (0/1): {neg_to_pos_ratio:.2f}")

✅ Data loaded successfully: (9809, 292)

--- Data Setup ---
Window: 5, Threshold: 0.005
Features: 285, Samples: 8372
Class imbalance ratio (0/1): 1.18


C:\Users\epoch_bpjmdqk\AppData\Local\Temp\ipykernel_12168\2980044324.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_features.replace([np.inf, -np.inf], np.nan, inplace=True)


In [4]:
# --- 2. THREE-WAY TIME SERIES SPLIT ---
print(f"\n--- Time Series Split ---")
train_mask = X_features.index <= CONFIG['train_end']
val_mask = (X_features.index > CONFIG['train_end']) & (X_features.index <= CONFIG['val_end'])
test_mask = X_features.index > CONFIG['val_end']

X_train = X_features[train_mask]
X_val = X_features[val_mask] 
X_test = X_features[test_mask]
y_train = y[train_mask]
y_val = y[val_mask]
y_test = y[test_mask]
returns_train = returns_full[train_mask]
returns_val = returns_full[val_mask]
returns_test = returns_full[test_mask]

print(f"Train: {len(X_train)} samples ({X_train.index.min()} to {X_train.index.max()})")
print(f"Val:   {len(X_val)} samples ({X_val.index.min()} to {X_val.index.max()})")
print(f"Test:  {len(X_test)} samples ({X_test.index.min()} to {X_test.index.max()})")



--- Time Series Split ---
Train: 7306 samples (1992-01-02 00:00:00 to 2020-12-31 00:00:00)
Val:   252 samples (2021-01-04 00:00:00 to 2021-12-31 00:00:00)
Test:  814 samples (2022-01-03 00:00:00 to 2025-04-01 00:00:00)


In [ ]:
# --- 3. FEATURE STABILITY ANALYSIS ---
stable_features = check_feature_stability(X_train, y_train, X_train.columns, CONFIG['random_state'])
print(f"\nUsing {len(stable_features)} most stable features for modeling")

# Filter to stable features
X_train_stable = X_train[stable_features]
X_val_stable = X_val[stable_features]
X_test_stable = X_test[stable_features]


--- Feature Stability Analysis ---
Most stable features (CV < 0.5):
  Open_^GSPC: CV = 0.123

Using 1 most stable features for modeling


In [6]:
# --- 4. REGIME DETECTION ---
market_returns = returns_full  # Or use market index if available
regimes = detect_market_regimes(market_returns)
print(f"\nMarket regimes detected:")
print(regimes.value_counts().sort_index())


Market regimes detected:
0.0    2089
1.0    3436
2.0    2847
Name: count, dtype: int64


In [7]:
# --- 5. ENHANCED OPTUNA OPTIMIZATION ---
def enhanced_objective(trial):
    # Expanded search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'scale_pos_weight': neg_to_pos_ratio,
        'early_stopping_rounds': 20,  # Move to model params
        'eval_metric': 'logloss',
        'random_state': CONFIG['random_state'],
        'n_jobs': -1,
        'tree_method': 'hist'
    }
    
    # Tunable pipeline components - limit PCA to available features
    max_pca_components = min(10, len(stable_features) - 1, 50)  # Conservative limit
    pca_components = trial.suggest_int('pca_n_components', 2, max_pca_components)
    selector_threshold = trial.suggest_categorical('selector_threshold', ['mean', 'median', '0.75*mean'])
    
    model = XGBClassifier(**params)
    selector = SelectFromModel(model, threshold=selector_threshold)
    
    # Time series cross-validation on training data only
    tscv = TimeSeriesSplit(n_splits=3)
    sharpe_scores = []
    
    for train_idx, val_idx in tscv.split(X_train_stable):
        X_cv_train = X_train_stable.iloc[train_idx]
        X_cv_val = X_train_stable.iloc[val_idx]
        y_cv_train = y_train.iloc[train_idx]
        y_cv_val = y_train.iloc[val_idx]
        returns_cv_val = returns_train.iloc[val_idx]
        
        try:
            # Fit feature selection and scaling first
            scaler = StandardScaler()
            X_cv_train_scaled = scaler.fit_transform(X_cv_train)
            X_cv_val_scaled = scaler.transform(X_cv_val)
            
            # Feature selection
            selector_temp = SelectFromModel(XGBClassifier(n_estimators=50, random_state=42), 
                                          threshold=selector_threshold)
            X_cv_train_selected = selector_temp.fit_transform(X_cv_train_scaled, y_cv_train)
            X_cv_val_selected = selector_temp.transform(X_cv_val_scaled)
            
            # Skip if too few features selected
            if X_cv_train_selected.shape[1] < 2:
                continue
                
            # PCA - adjust components if needed
            actual_pca_components = min(pca_components, X_cv_train_selected.shape[1])
            pca = PCA(n_components=actual_pca_components)
            X_cv_train_pca = pca.fit_transform(X_cv_train_selected)
            X_cv_val_pca = pca.transform(X_cv_val_selected)
            
            # Train final model with early stopping
            model_temp = XGBClassifier(**{k: v for k, v in params.items() 
                                        if k not in ['early_stopping_rounds']})
            model_temp.fit(X_cv_train_pca, y_cv_train,
                          early_stopping_rounds=20,
                          eval_set=[(X_cv_val_pca, y_cv_val)],
                          verbose=False)
            
            preds = model_temp.predict(X_cv_val_pca)
            strategy_returns = preds * returns_cv_val
            sharpe = calculate_sharpe_with_costs(strategy_returns, CONFIG['transaction_cost'])
            sharpe_scores.append(sharpe if not np.isnan(sharpe) and np.isfinite(sharpe) else -1)
            
        except Exception as e:
            # Silently skip failed trials during optimization
            continue
    
    return np.mean(sharpe_scores) if sharpe_scores else -10

print("\n--- Starting Optuna Optimization ---")

# Run Optuna study
study = optuna.create_study(direction='maximize', 
                          sampler=optuna.samplers.TPESampler(seed=CONFIG['random_state']))
study.optimize(enhanced_objective, n_trials=CONFIG['n_trials'], show_progress_bar=True)

print(f"\n--- Optimization Results ---")
print(f"Best Sharpe ratio: {study.best_value:.4f}")
print(f"Best parameters: {study.best_params}")

best_params = study.best_params.copy()

# Extract pipeline parameters
pca_components = best_params.pop('pca_n_components')
selector_threshold = best_params.pop('selector_threshold')

[I 2025-08-29 17:31:22,322] A new study created in memory with name: no-name-6219e6e9-aa18-458c-b447-ace441e87f7b



--- Starting Optuna Optimization ---


  0%|          | 0/100 [00:00<?, ?it/s]

[W 2025-08-29 17:31:22,336] Trial 0 failed with parameters: {'n_estimators': 362, 'learning_rate': 0.17254716573280354, 'max_depth': 7, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'gamma': 0.7799726016810132} because of the following error: ValueError('The `low` value must be smaller than or equal to the `high` value (low=2, high=0).').
Traceback (most recent call last):
  File "c:\Users\epoch_bpjmdqk\Documents\Code\venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\epoch_bpjmdqk\AppData\Local\Temp\ipykernel_12168\3573687207.py", line 21, in enhanced_objective
    pca_components = trial.suggest_int('pca_n_components', 2, max_pca_components)
  File "c:\Users\epoch_bpjmdqk\Documents\Code\venv\Lib\site-packages\optuna\_convert_positional_args.py", line 135, in converter_wrapper
    return func(**kwargs)  # type: ignore[call-arg]
  File "c:\Users\epoch_bpjmdqk\Documents\Code\venv\Lib\sit

ValueError: The `low` value must be smaller than or equal to the `high` value (low=2, high=0).

In [ ]:
# --- 6. TRAIN FINAL MODEL ---
print("\n--- Training Final Model ---")

# Extract best parameters and remove non-XGBoost params
final_model_params = {k: v for k, v in best_params.items() 
                      if k not in ['pca_n_components', 'selector_threshold']}
final_model_params.update({
    'scale_pos_weight': neg_to_pos_ratio,
    'early_stopping_rounds': 20,
    'eval_metric': 'logloss', 
    'random_state': CONFIG['random_state'],
    'n_jobs': -1,
    'tree_method': 'hist'
})

# Build pipeline step by step for better control
scaler = StandardScaler()
temp_model = XGBClassifier(n_estimators=50, random_state=42)  # Quick model for feature selection
selector = SelectFromModel(temp_model, threshold=selector_threshold)

# Fit preprocessing steps
X_train_scaled = scaler.fit_transform(X_train_stable)
X_val_scaled = scaler.transform(X_val_stable)

# Feature selection
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_val_selected = selector.transform(X_val_scaled)

print(f"Features after selection: {X_train_selected.shape[1]}")

# PCA with dynamic component adjustment
actual_pca_components = min(pca_components, X_train_selected.shape[1])
pca = PCA(n_components=actual_pca_components)
X_train_pca = pca.fit_transform(X_train_selected)
X_val_pca = pca.transform(X_val_selected)

print(f"PCA components used: {actual_pca_components}")

# Train final model with early stopping
final_model = XGBClassifier(**final_model_params)
final_model.fit(X_train_pca, y_train,
               early_stopping_rounds=20,
               eval_set=[(X_val_pca, y_val)],
               verbose=True)

# Create complete pipeline for saving
final_pipeline = Pipeline([
    ('scaler', scaler),
    ('feature_select', selector), 
    ('pca', pca),
    ('model', final_model)
])

print("✅ Final model trained with early stopping")

In [ ]:
# --- 7. COMPREHENSIVE MODEL EVALUATION ---
def evaluate_model_comprehensive(scaler, selector, pca_model, model, X_test, y_test, returns_test, regimes_test, set_name="Test"):
    """Comprehensive model evaluation with manual pipeline"""
    print(f"\n--- {set_name} Set Evaluation ---")
    
    # Manual pipeline prediction
    X_test_scaled = scaler.transform(X_test)
    X_test_selected = selector.transform(X_test_scaled)
    X_test_pca = pca_model.transform(X_test_selected)
    
    # Basic predictions
    y_pred = model.predict(X_test_pca)
    y_pred_proba = model.predict_proba(X_test_pca)[:, 1]
    
    # Strategy returns
    strategy_returns = y_pred * returns_test
    
    # Performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    sharpe = calculate_sharpe_with_costs(strategy_returns, CONFIG['transaction_cost'])
    max_dd = calculate_max_drawdown(strategy_returns.cumsum())
    total_return = strategy_returns.sum()
    
    # Win rate and trade statistics
    winning_trades = strategy_returns[strategy_returns > 0]
    losing_trades = strategy_returns[strategy_returns < 0]
    win_rate = len(winning_trades) / len(strategy_returns[strategy_returns != 0]) if len(strategy_returns[strategy_returns != 0]) > 0 else 0
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Sharpe Ratio (w/ costs): {sharpe:.4f}")
    print(f"Total Return: {total_return:.4f}")
    print(f"Max Drawdown: {max_dd:.4f}")
    print(f"Win Rate: {win_rate:.4f}")
    print(f"Avg Winning Trade: {winning_trades.mean():.6f}")
    print(f"Avg Losing Trade: {losing_trades.mean():.6f}")
    print(f"Number of Trades: {(y_pred != 0).sum()}")
    
    # Regime-specific performance
    if regimes_test is not None and len(regimes_test) > 0:
        print(f"\nRegime Performance:")
        for regime in regimes_test.unique():
            regime_mask = regimes_test == regime
            if regime_mask.sum() > 10:
                regime_returns = strategy_returns[regime_mask]
                regime_sharpe = calculate_sharpe_with_costs(regime_returns, CONFIG['transaction_cost'])
                regime_names = {0: 'Low Vol', 1: 'Normal Vol', 2: 'High Vol'}
                print(f"  {regime_names.get(regime, f'Regime {regime}')}: Sharpe = {regime_sharpe:.3f} ({regime_mask.sum()} samples)")
    
    return {
        'accuracy': accuracy,
        'sharpe': sharpe,
        'total_return': total_return,
        'max_drawdown': max_dd,
        'win_rate': win_rate,
        'n_trades': (y_pred != 0).sum()
    }

# Evaluate on validation set
val_regimes = regimes[val_mask] if len(regimes[val_mask]) > 0 else None
val_results = evaluate_model_comprehensive(scaler, selector, pca, final_model, X_val_stable, y_val, 
                                         returns_val, val_regimes, "Validation")

# Evaluate on test set (final, unbiased evaluation)
test_regimes = regimes[test_mask] if len(regimes[test_mask]) > 0 else None
test_results = evaluate_model_comprehensive(scaler, selector, pca, final_model, X_test_stable, y_test, 
                                          returns_test, test_regimes, "Test")

In [ ]:
# --- 8. ROBUST PERMUTATION IMPORTANCE ---
print("\n--- Permutation Importance Analysis ---")

# Get selected features from the fitted selector
selected_features = X_train_stable.columns[selector.get_support()]
print(f"Selected {len(selected_features)} features for importance analysis")

# Use only training data for feature importance to avoid bias
# Note: We use the original selected features, not PCA components
X_train_for_pi = X_train_stable[selected_features]
pi_result = permutation_importance(
    lambda X: final_model.predict(pca.transform(selector.transform(scaler.transform(X)))),
    X_train_for_pi, y_train, 
    n_repeats=10, random_state=CONFIG['random_state'], n_jobs=-1,
    scoring='accuracy'  # Use accuracy since we need a scorer function
)

sorted_idx = pi_result.importances_mean.argsort()[::-1]

print("Top 10 Most Important Features:")
for i in sorted_idx[:10]:
    if i < len(selected_features):
        feature_name = selected_features[i]
        importance_mean = pi_result.importances_mean[i]
        importance_std = pi_result.importances_std[i]
        print(f"  {feature_name}: {importance_mean:.4f} ± {importance_std:.4f}")

In [ ]:
# --- 9. OVERFITTING CHECKS ---
print("\n--- Overfitting Analysis ---")

# Compare train vs validation performance using manual pipeline
X_train_scaled_eval = scaler.transform(X_train_stable)
X_train_selected_eval = selector.transform(X_train_scaled_eval)
X_train_pca_eval = pca.transform(X_train_selected_eval)
train_pred = final_model.predict(X_train_pca_eval)
train_strategy_returns = train_pred * returns_train
train_sharpe = calculate_sharpe_with_costs(train_strategy_returns, CONFIG['transaction_cost'])

print(f"Training Sharpe: {train_sharpe:.4f}")
print(f"Validation Sharpe: {val_results['sharpe']:.4f}")
print(f"Test Sharpe: {test_results['sharpe']:.4f}")

sharpe_degradation = (train_sharpe - test_results['sharpe']) / abs(train_sharpe) if train_sharpe != 0 else 0
print(f"Sharpe degradation (train→test): {sharpe_degradation:.2%}")

if sharpe_degradation > 0.5:
    print("Warning: Significant performance degradation detected. Model may be overfitting.")
elif sharpe_degradation > 0.2:
    print("Caution: Moderate performance degradation. Monitor in backtesting.")
else:
    print("Performance degradation within acceptable limits.")

In [ ]:
# --- 10. MODEL PERSISTENCE ---
os.makedirs(CONFIG['model_dir'], exist_ok=True)

# Save the final pipeline
model_filename = os.path.join(CONFIG['model_dir'], "final_xgb_optuna_pipeline_enhanced.pkl")
with open(model_filename, 'wb') as f:
    pickle.dump(final_pipeline, f)

# Save model metadata
metadata = {
    'config': CONFIG,
    'best_params': {**final_model_params, 'pca_n_components': actual_pca_components, 'selector_threshold': selector_threshold},
    'class_imbalance_ratio': neg_to_pos_ratio,
    'selected_features': list(selected_features),
    'stable_features': stable_features,
    'validation_results': val_results,
    'test_results': test_results,
    'train_sharpe': train_sharpe,
    'feature_importance': {
        feature: float(importance) 
        for i, (feature, importance) in enumerate(zip(selected_features[sorted_idx[:20]], 
                                     pi_result.importances_mean[sorted_idx[:20]]))
        if i < len(selected_features)
    },
    'training_date': datetime.now().isoformat()
}

metadata_filename = os.path.join(CONFIG['model_dir'], "model_metadata_enhanced.json")
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"\n✅ Enhanced pipeline saved as '{model_filename}'")
print(f"✅ Model metadata saved as '{metadata_filename}'")

In [ ]:
# --- 11. REGIME-AWARE FINAL VALIDATION ---
if len(regimes) > 0:
    print("\n--- Final Regime Analysis ---")
    regime_performance = regime_aware_validation(X_test_stable, y_test, returns_test, test_regimes, final_pipeline)

print("\n--- Summary ---")
print(f"Final Test Sharpe Ratio: {test_results['sharpe']:.4f}")
print(f"Final Test Accuracy: {test_results['accuracy']:.4f}")
print(f"Model ready for backtesting phase.")